In [1]:
# !pip install web3
# !pip install python-dotenv
# !pip install requests

## Imports

In [2]:
from web3 import Web3

# To read environment property file
import os
from dotenv import load_dotenv
from pathlib import Path

# To load ABI
import json

# For Http calls
import requests

# Import math Library
import math

## Constants

In [3]:
# Uniswap v3 Factory address
UNISWAP_V3_FACTORY = '0x1F98431c8aD98523631AE4a59f267346ea31F984'

# Uniswap v3 NFT Manager
NFT_POSITION_MANAGER = '0xC36442b4a4522E871399CD717aBDD847Ab11FE88'

# Etherscan endpoint
ETHERSCAN_ENDPOINT = 'https://api.etherscan.io/api'

# Uniswap v3 Pool  ABI path
POOL_ABI = 'abi/uniswap_pool_abi.json'

# Path to the ERC20 ABI
ERC20_ABI = 'abi/erc20.json'

# Constants for Uniswap maths
Q96 = 2 ** 96
MAX_UINT128 = 2**128 - 1

## Load environment variables

In [4]:
dotenv_path = Path('.env/nft_amounts')
load_dotenv(dotenv_path=dotenv_path)
PROVIDER_URL = os.getenv('PROVIDER_URL')
ETHERSCAN_API_KEY = os.getenv('ETHERSCAN_API_KEY')

## Method to get Uniswap v3 factory

In [5]:
def get_abi(address:str) -> str:
    """ Returns the ABI for given address
        
    Returns:
    str
        The factory ABI or None for errors
    """
    # Get ABI of contract; use etherscan API for verifiable smart contracts ABI
    params = {'module':'contract', 'action':'getabi', 'address':{address}, 'apikey':{ETHERSCAN_API_KEY}}
    response = json.loads(requests.get(url=ETHERSCAN_ENDPOINT, params=params).text)
    if response['status'] == '1':
        return json.loads(response['result'])
    return None

In [6]:
# Instantiate web3 instance for us to interact with the chain
web3 = Web3(Web3.HTTPProvider(PROVIDER_URL))

# NFT ABI
nft_abi = get_abi(address=NFT_POSITION_MANAGER)
assert nft_abi != None, 'NFT Position Manager ABI not available'

# Create nft contract
nft_contract = web3.eth.contract(address=NFT_POSITION_MANAGER, abi=nft_abi)
assert nft_contract != None, 'NFT contract does not exist'

# Uniswap Factory ABI
factory_abi = get_abi(address=UNISWAP_V3_FACTORY)
assert factory_abi != None, 'Factory ABI not available'

# Create factory contract
factory_contract = web3.eth.contract(address=UNISWAP_V3_FACTORY, abi=factory_abi)
assert factory_contract != None, 'Facory contract does not exist'

# Load the previously downloaded pool ABI
with open(POOL_ABI) as f:
    pool_abi = json.load(f)
assert pool_abi != None, 'Pool ABI not available'

# Load ERC20 ABIs to get decimails for token0 and token1
with open(ERC20_ABI) as f:
    erc20_abi = json.load(f)
assert erc20_abi != None, 'ERC20 ABI not available'

In [7]:
# Token ID of interest
token_id = 952381

# Token position details
_,_,token0,token1,fees,tick_lower,tick_upper,\
    liquidity,_,_,_,_, = nft_contract.functions.positions(token_id).call()

# No point going further if the liquidity is zero
assert liquidity != 0, 'Liquidity must not be zero'

# Get pool contract for token0, token1 and fees
pool_address = factory_contract.functions.getPool(token0, token1, fees).call()
pool_contract = web3.eth.contract(address=pool_address, abi=pool_abi)
assert pool_contract != None, 'Pool contract does not exist for token0 and token1'

# Get sqrtPriceX96 from pool contract
sqrt_priceX96 = pool_contract.functions.slot0().call()[0]

# Get token0 decimals
token0_address = pool_contract.functions.token0().call()
token0_contract = web3.eth.contract(address=token0_address, abi=erc20_abi)
decimal0 = token0_contract.functions.decimals().call()

# Get token1 decimals
token1_address = pool_contract.functions.token1().call()
token1_contract = web3.eth.contract(address=token1_address, abi=erc20_abi)
decimal1 = token1_contract.functions.decimals().call()

In [8]:
def convert_wei_to_human(amount:int, decimals:int) -> int:
    """ Converts given amount to human friendly format
    Parameters:
    amount: 
        amoutn to convert
    decimals : int
        number of decimals for the token

    Returns:
    int
        Token amount converted to human friendly format
    """    
    return round((amount/(10**decimals)), decimals)

## Return the token amounts
### Reference: [A Primer on Uniswap v3 Math Part 2: Stay Awake by Reading it Aloud](https://blog.uniswap.org/uniswap-v3-math-primer-2)

In [9]:
def get_token_amounts(liquidity:int, sqrtx96:int, tick_low:int, tick_high:int) -> tuple:
    """ Calculates and returns token amounts
    Parameters:
    liquidity: int
        Liquidity from the NFT position
    sqrtx96 : int
        from the pool's slot0 call
    tick_low: int
        low tick position rfom 
    tick_high : int
        high tick position

    Returns:
    tuple
        A tuple consisting of token0 and token1 amounts or zero if the liquidity is zero
    """    
    sqrt_ratio_l = math.sqrt(1.0001**tick_low)
    sqrt_ratio_h = math.sqrt(1.0001**tick_high)
    current_tick = math.floor(math.log((sqrtx96/Q96)**2)/math.log(1.0001))
    sqrt_price = sqrt_priceX96/Q96
    amount0 = 0
    amount1 = 0
    if current_tick < tick_low:
        amount0 = math.floor(liquidity*((sqrt_ratio_h - sqrt_ratio_l)/(sqrt_ratio_l * sqrt_ratio_h)))
    elif current_tick >= tick_high:
        amount1 = math.floor(liquidity*(sqrt_ratio_h - sqrt_ratio_l))
    elif current_tick >= tick_low and current_tick < tick_high:
        amount0 = math.floor(liquidity*((sqrt_ratio_h - sqrt_price)/(sqrt_price * sqrt_ratio_h)))
        amount1 = math.floor(liquidity*(sqrt_price - sqrt_ratio_l))

    return (amount0, amount1)

In [10]:
amount0, amount1 = get_token_amounts(liquidity, sqrt_priceX96, tick_lower, tick_upper)
print('Amount - Token0: {amount0} Token1: {amount1}'.format(
    amount0=convert_wei_to_human(amount0, decimal0), amount1=convert_wei_to_human(amount1, decimal1)))

Amount - Token0: 2.09385807323503 Token1: 29259.694666848405


## Ranges
### Reference: [LIQUIDITY MATH IN UNISWAP V3](https://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf)

## Range (token0 in terms of token1)

In [11]:
price_a = (1.0001**tick_lower)
price_b = (1.0001**tick_upper)
# Adjust them
price_a_adj = price_a * (10 **(decimal0-decimal1))
price_b_adj = price_b * (10 **(decimal0-decimal1))
print('Range (token0 in terms of token1) - [{:.5f} - {:.5f}]'.format(price_a_adj, price_b_adj))

Range (token0 in terms of token1) - [18760.48421 - 46141.26906]


## Range (token1 in terms of token0)

In [12]:
price_a = (1.0001**tick_lower)
price_b = (1.0001**tick_upper)

# Adjust them
price_a_adj = price_a * (10 **(decimal0-decimal1))
price_b_adj = price_b * (10 **(decimal0-decimal1))

# Invert to get token1 in terms of token0
price_a_adj = 1/price_a_adj
price_b_adj = 1/price_b_adj
print('Range (token1 in terms of token0) - [{:.5f} - {:.5f}]'.format(price_b_adj, price_a_adj))

Range (token1 in terms of token0) - [0.00002 - 0.00005]


## Unclaimed Fees

In [13]:
# We need to get the owner for the token first
owner_address = nft_contract.functions.ownerOf(tokenId=token_id).call()
collect_dict = {'tokenId' : token_id, 'recipient': owner_address, 'amount0Max': MAX_UINT128, 'amount1Max': MAX_UINT128}
amount0, amount1 = nft_contract.functions.collect(collect_dict).call()
print('Uncollected Fees - Token0: {amount0} Token1: {amount1}'.format(
    amount0=convert_wei_to_human(amount0, decimal0), amount1=convert_wei_to_human(amount1, decimal1)))

Uncollected Fees - Token0: 0.011358290111891906 Token1: 248.03215759946923
